In [1]:
!pip install openai-whisper --quiet
!pip install streamlit --quiet
!pip install pypdub --quiet
!pip install ffmpeg-python --quiet
!pip install torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.7 MB/s

In [2]:
!pip install soundfile --quiet

In [3]:
!pip install gtts --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.3 MB/s eta 0:00:00


In [4]:
import whisper

In [5]:
model=whisper.load_model("base")
audio_file=model.transcribe("/content/ai_audio.mp3")
print(audio_file['text'])

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 81.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Artificial intelligence, often abbreviated as AI, refers to the simulation of human intelligence by machines. It enables computers and systems to perform tasks such as learning, reasoning, problem solving, and language understanding. Today, AI powers technologies like voice assistance, recommendation engines, chatbots, self-driving cars, and more. Machine learning, a subset of AI, allows systems to improve automatically through experience. While AI offers great promise in fields like healthcare, finance, and education, it also raises important questions about ethics, data privacy, and job automation. As AI continues to evolve, it's crucial that we develop and deploy it responsibly to benefit humanity.


In [6]:
for segment in audio_file["segments"]:
    print(f"[{segment['start']:.2f} - {segment['end']:.2f}] {segment['text']}")


[0.00 - 9.00]  Artificial intelligence, often abbreviated as AI, refers to the simulation of human intelligence by machines.
[9.00 - 18.20]  It enables computers and systems to perform tasks such as learning, reasoning, problem solving, and language understanding.
[18.20 - 27.80]  Today, AI powers technologies like voice assistance, recommendation engines, chatbots, self-driving cars,
[27.80 - 36.30]  and more. Machine learning, a subset of AI, allows systems to improve automatically through experience.
[36.30 - 49.40]  While AI offers great promise in fields like healthcare, finance, and education, it also raises important questions about ethics, data privacy, and job automation.
[49.40 - 57.40]  As AI continues to evolve, it's crucial that we develop and deploy it responsibly to benefit humanity.


In [7]:
!pip install noisereduce --quiet

In [8]:
import noisereduce as nr
import soundfile as sf

In [9]:
data, rate=sf.read("/content/ai_audio.mp3")
reduce_noise=nr.reduce_noise(y=data,sr=rate)

In [10]:
sf.write("cleaned_audio.wav", reduce_noise, rate)

In [13]:
!pip install flask_ngrok --quiet

In [17]:
!pip install -q flask pyngrok openai-whisper noisereduce soundfile librosa


In [20]:
from flask import Flask, request, render_template_string
import threading
import tempfile
import os
import whisper
import librosa
import soundfile as sf
import noisereduce as nr
from pyngrok import ngrok

# Load Whisper model
model = whisper.load_model("base")

# Set ngrok auth token
ngrok.set_auth_token("2xglfrOOwfqYTSWdzUdwjrHHAeh_6Ug9uQKqdMjrfQEvfpdEL")

# Flask app setup
app = Flask(__name__)

HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Whisper Transcription</title>
</head>
<body>
    <h2>Upload Audio File</h2>
    <form method="post" enctype="multipart/form-data">
        <input type="file" name="audio_file" accept="audio/*" required><br><br>
        <label><input type="checkbox" name="noise_reduce"> Apply Noise Reduction</label><br><br>
        <input type="submit" value="Transcribe">
    </form>

    {% if transcript %}
    <h3>Transcript:</h3>
    <p>{{ transcript }}</p>
    <h4>Segments:</h4>
    <ul>
        {% for segment in segments %}
        <li><strong>[{{ segment.start | round(2) }} - {{ segment.end | round(2) }}]</strong>: {{ segment.text }}</li>
        {% endfor %}
    </ul>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    transcript = None
    segments = []

    if request.method == "POST":
        file = request.files["audio_file"]
        reduce_noise = 'noise_reduce' in request.form

        y, sr = librosa.load(file, sr=None)
        if reduce_noise:
            y = nr.reduce_noise(y=y, sr=sr)

        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp:
            sf.write(temp.name, y, sr)
            audio_path = temp.name

        result = model.transcribe(audio_path)
        transcript = result["text"]
        segments = result["segments"]

        os.remove(audio_path)

    return render_template_string(HTML, transcript=transcript, segments=segments)

# Start Flask in thread
def run_flask():
    app.run(port=7860)

thread = threading.Thread(target=run_flask)
thread.start()

# Connect ngrok to port 7860
public_url = ngrok.connect(7860)
print("Your app is live at:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:7860
INFO:werkzeug:Press CTRL+C to quit


Your app is live at: NgrokTunnel: "https://518f-34-58-56-237.ngrok-free.app" -> "http://localhost:7860"
